  # Lab: Acceso a modelos open source de IA Generativa


  ### Objetivos del Lab

  - Aprender a usar 
    - LM Studio
      - Como herramienta visual (GUI)
      - Desde Python
    - Ollama
        - Desde la terminal (CLI)
        - Desde Python

  - Integrar un modelo local en nuestro Asistente financiero



  ### Herramientas

  - **LM Studio**: Interfaz gráfica

  - **Ollama**: CLI para desarrolladores



  ### Estructura del Lab

  1. **Preparación del entorno**

  2. **LM Studio**

  3. **Ollama**

  4. **Modelos de Hugging Face desde Ollama**

  5. **Comparación: Local vs Cloud**

  6. **Problema: Asistente Financiero Local**

  ## 1. Preparación del Entorno







  ### Instalación de dependencias







  Ejecuta la siguiente celda para instalar las librerías necesarias:

In [ ]:
# Instalar dependencias para modelos open source
!pip install ollama lmstudio python-dotenv google-genai openai 


  ### Instalación de herramientas



  #### LM Studio



  1. Descarga desde [lmstudio.ai](https://lmstudio.ai/download)

  2. Instala la aplicación para tu sistema operativo

  3. Ejecuta LM Studio



  #### Ollama



  1. Descarga desde [ollama.com](https://ollama.com/download)

  2. Instala según tu sistema operativo

  3. Verifica instalación ejecutando `ollama --version` en terminal



  ### 🔧 Verificación de instalación



  **LM Studio**: Abre la aplicación y verifica que aparezca la interfaz

  **Ollama**: Ejecuta `ollama --version` en terminal



  ## 2. LM Studio - La Forma Visual







  ### Características principales:



  - **Interfaz gráfica intuitiva**: Ideal para principiantes

  - **Gestión visual de modelos**: Descarga y organización sencilla

  - **Servidor local integrado**: API compatible con OpenAI

  - **Soporte multiplataforma**: Windows, macOS, Linux







  ### Pasos para usar LM Studio:



  1. **Abre LM Studio**



  2. **Busca un modelo** (ej: "qwen2.5:0.5b")



  3. **Descarga el modelo**



  4. **Inicia el servidor local**



  5. **Conecta desde Python**







  💡 **Consejo**: Empieza con modelos pequeños como `qwen2.5:0.5b` para pruebas rápidas.

  ### 📚 Documentación útil:



  - [LM Studio Official Site](https://lmstudio.ai/)


  - [LM Studio Documentation](https://lmstudio.ai/docs/app)


  - [LM Studio Python SDK](https://github.com/lmstudio-ai/lmstudio-python)



  ### Opción 1: SDK Nativo de LM Studio



  **Ventajas**: Funciones específicas, streaming avanzado, gestión completa

In [ ]:
import lmstudio as lms
import json
import pprint
from lmstudio import LlmPredictionConfig

def usar_lm_studio_nativo(mensaje, temperatura=0.7):
    """Usar SDK nativo de LM Studio"""
    
    try:
        # 1. Obtener modelo 
        model = lms.llm("qwen2.5-0.5b-instruct")
        #client = lms.get_default_client()
        #model = client.llm.load_new_instance("qwen2.5-0.5b-instruct")  
        
        # 2. Crear contexto de chat
        chat = lms.Chat("Eres un asistente útil y amigable.")
        chat.add_user_message(mensaje)

        config = LlmPredictionConfig(
            temperature=temperatura,
            max_tokens=1
        )
        
        # 3. Generar respuesta
        result = model.respond(chat, config=config)
        
        print("🖥️ LM Studio responde:")
        print(result.content)
        
        print("Stats:")
        print(result.stats)  # Mostrar detalles del resultado

        
        # 4. Información adicional
        print(f"\n📊 Tokens generados: {result.stats.predicted_tokens_count}")
        print(f"⚡ Tiempo al primer token: {result.stats.time_to_first_token_sec:.2f}s")
        print(f"🛑 Razón de parada: {result.stats.stop_reason}")
        
        return result
        
    except Exception as e:
        print(f"❌ Error con SDK nativo de LM Studio: {e}")
        print("💡 Asegúrate de que LM Studio esté corriendo")
        return None

def usar_lm_studio_streaming(mensaje):
    """Demostrar streaming con SDK nativo"""
    
    try:
        # 1. Obtener modelo
        model = lms.llm("qwen2.5-0.5b-instruct")
        
        # 2. Crear contexto
        chat = lms.Chat("Eres un asistente útil y amigable.")
        chat.add_user_message(mensaje)
        
        # 3. Streaming
        print(f"🖥️ LM Studio (streaming) responde:")
        print("🖥️ Asistente: ", end="", flush=True)
        result = model.respond_stream(chat) 
        for fragment in result:
            print(fragment.content, end="", flush=True)
        
        print()  # Nueva línea al final
        
    except Exception as e:
        print(f"❌ Error con streaming: {e}")

# Probar SDK nativo
#usar_lm_studio_nativo("¿Cuál es la capital de España?")
usar_lm_studio_streaming("Explica brevemente qué es la inteligencia artificial")


  ### Opción 2: SDK de OpenAI



  **Ventajas**: Estándar de industria, código reutilizable, compatible con todos los proveedores

In [ ]:
import os
from openai import OpenAI
from dotenv import load_dotenv

# Cargar variables de entorno
load_dotenv()

def usar_lm_studio_openai(mensaje, temperatura=0.7):
    """Conectar con LM Studio usando SDK de OpenAI"""
    
    # 1. Configurar cliente para LM Studio
    client = OpenAI(
        base_url="http://localhost:1234/v1",  # Servidor local de LM Studio
        api_key=""  
    )
    
    try:
        # 2. Hacer llamada estándar
        response = client.chat.completions.create(
            model="",  # Nombre del modelo en LM Studio
            messages=[
                {"role": "user", "content": mensaje}
            ],
            temperature=temperatura,
            max_tokens=150
        )
        
        # 3. Extraer respuesta
        contenido = response.choices[0].message.content
        
        print(f"🖥️ LM Studio responde:")
        print(contenido)
        
        # 4. Información adicional
        print(f"\n📊 Tokens usados: {response.usage.total_tokens}")
        print(f"🏠 Ejecutándose localmente - sin coste")
        
        return contenido
        
    except Exception as e:
        print(f"❌ Error conectando con LM Studio: {e}")
        print("💡 Asegúrate de que LM Studio esté corriendo con el servidor iniciado")
        return None

# Probar conexión con LM Studio
usar_lm_studio_openai("¿Cuál es la capital de España?")


  ### Funciones útiles del SDK Nativo

In [ ]:
import lmstudio as lms

def listar_modelos_cargados():
    """Listar modelos actualmente cargados en memoria"""
    
    try:
        # Listar todos los modelos cargados
        todos_modelos = lms.list_loaded_models()
        modelos_llm = lms.list_loaded_models("llm")
        modelos_embedding = lms.list_loaded_models("embedding")
        
        print("📋 Modelos cargados en memoria:")
        print("=" * 50)
        
        if not todos_modelos:
            print("❌ No hay modelos cargados en memoria")
            print("💡 Carga un modelo desde la interfaz de LM Studio")
            return None
        
        print(f"🔄 Total de modelos cargados: {len(todos_modelos)}")
        print(f"🤖 Modelos LLM: {len(modelos_llm)}")
        print(f"🔍 Modelos embedding: {len(modelos_embedding)}")
        print()
        
        # Mostrar detalles de cada modelo
        for i, modelo in enumerate(todos_modelos, 1):
            print(f"{i}. {modelo}")
        
        return {
            'todos': todos_modelos,
            'llm': modelos_llm,
            'embedding': modelos_embedding
        }
        
    except Exception as e:
        print(f"❌ Error listando modelos cargados: {e}")
        print("💡 Asegúrate de que LM Studio esté ejecutándose")
        return None

# Ejecutar función
listar_modelos_cargados()


  ### 2. Listar modelos descargados localmente

In [ ]:
def listar_modelos_descargados():
    """Listar modelos disponibles localmente en LM Studio"""
    
    try:
        # Listar todos los modelos descargados
        todos_descargados = lms.list_downloaded_models()
        llm_descargados = lms.list_downloaded_models("llm")
        embedding_descargados = lms.list_downloaded_models("embedding")
        
        print("📋 Modelos descargados localmente:")
        print("=" * 50)
        
        if not todos_descargados:
            print("❌ No hay modelos descargados")
            print("💡 Descarga modelos desde la interfaz de LM Studio")
            return None
        
        print(f"📦 Total de modelos descargados: {len(todos_descargados)}")
        print(f"🤖 Modelos LLM: {len(llm_descargados)}")
        print(f"🔍 Modelos embedding: {len(embedding_descargados)}")
        print()
        
        # Mostrar detalles de cada modelo descargado
        for i, modelo in enumerate(todos_descargados, 1):
            print(f"{i}. {modelo}")
        
        return {
            'todos': todos_descargados,
            'llm': llm_descargados,
            'embedding': embedding_descargados
        }
        
    except Exception as e:
        print(f"❌ Error listando modelos descargados: {e}")
        print("💡 Asegúrate de que LM Studio esté ejecutándose")
        return None

# Ejecutar función
listar_modelos_descargados()


  ### Comparación: SDK Nativo vs OpenAI

  - **SDK OpenAI**: Para máxima compatibilidad y migración fácil

  - **SDK Nativo**: Para funciones avanzadas específicas de LM Studio

  ## 3. Ollama - La Forma CLI



  ### Características principales:



  - **Interfaz de línea de comandos**: Ideal para desarrolladores

  - **Gestión eficiente de modelos**: Comandos simples y potentes

  - **API REST integrada**: Compatible con OpenAI SDK



  ### Pasos para usar Ollama:



  1. **[Instalar Ollama](https://ollama.com/)**

  2. **Descargar modelo**: `ollama run gemma3:1b`

  3. **Verificar instalación**: `ollama list`

  4. **Conectar desde Python**


  📚 **Documentación útil:**

  - [Ollama Official Site](https://ollama.com/)

  - [Ollama Python SDK](https://github.com/ollama/ollama-python)

  - [Ollama Model Library](https://ollama.com/library)

  - [Parametros validos Python SDK](https://github.com/ollama/ollama/blob/main/docs/modelfile.md#valid-parameters-and-values)



  #### Opción 1: SDK Nativo de Ollama


In [ ]:
import ollama
import pprint

def usar_ollama_nativo(mensaje, temperatura=0.7):
    """Conectar con Ollama usando SDK nativo"""
    
    try:
        # 1. Llamada simple y directa
        response = ollama.chat(
            model="gemma3:1b", 
            messages=[
                {'role': 'user', 'content': mensaje}
            ],
            options={
                'temperature': temperatura,
                'num_predict': 150
            }
        )
        
        # 2. Extraer respuesta
        contenido = response['message']['content']
        
        print("🦙 Ollama responde:")
        print(contenido)

        print("\nDetalles de la respuesta:")
        pprint.pprint(response.model_dump())
        
        # 3. Información adicional
        if 'eval_count' in response:
            print(f"\n📊 Tokens generados: {response['eval_count']}")
        if 'eval_duration' in response:
            duracion = response['eval_duration'] / 1e9  # Convertir a segundos
            print(f"⚡ Tiempo de evaluación: {duracion:.2f}s")
        
        return contenido
        
    except Exception as e:
        print(f"❌ Error con SDK nativo de Ollama: {e}")

def usar_ollama_streaming(mensaje):
    """Demostrar streaming con SDK nativo de Ollama"""
    
    try:
        print("🦙 Ollama (streaming) responde:")
        print("🦙 Asistente: ", end="", flush=True)
        
        # 1. Streaming nativo
        stream = ollama.chat(
            model="gemma3:1b",
            messages=[{'role': 'user', 'content': mensaje}],
            stream=True
        )
        
        # 2. Procesar chunks
        for chunk in stream:
            if 'message' in chunk:
                contenido = chunk['message']['content']
                print(contenido, end="", flush=True)
        
        print()  # Nueva línea al final
        
    except Exception as e:
        print(f"❌ Error con streaming: {e}")

# Probar funciones
#usar_ollama_nativo("¿Cuál es la capital de España?")
usar_ollama_streaming("Explica brevemente qué es la inteligencia artificial")


  #### Opción 2: SDK de OpenAI



  **Ventajas**: Estándar de industria, código reutilizable, compatible con todos los proveedores

In [ ]:
import os
from openai import OpenAI
import pprint


def usar_ollama_con_openai_sdk(mensaje, temperatura=0.7):
    """Conectar con Ollama usando SDK de OpenAI"""
    
    # 1. Configurar cliente para Ollama
    client = OpenAI(
        base_url="http://localhost:11434/v1",  # Puerto estándar de Ollama
        api_key="ollama"  # API key dummy, Ollama no la requiere
    )
    
    try:
        # 2. Hacer llamada estándar
        response = client.chat.completions.create(
            model="gemma3:1b", 
            messages=[
                {"role": "user", "content": mensaje}
            ],
            temperature=temperatura,
            max_tokens=150
        )
        
        # 3. Extraer respuesta
        contenido = response.choices[0].message.content
        
        print(f"🦙 Ollama responde:")
        print(contenido)

        print("\nDetalles de la respuesta:")
        pprint.pprint(response.model_dump())
        
        # 4. Información adicional
        if hasattr(response, 'usage') and response.usage:
            print(f"\n📊 Tokens usados: {response.usage.total_tokens}")
        print(f"🏠 Ejecutándose localmente - sin coste")
        
        return contenido
        
    except Exception as e:
        print(f"❌ Error conectando con Ollama: {e}")
        return None

# Probar conexión con Ollama
usar_ollama_con_openai_sdk("¿Cuál es la capital de España?")


  ### Funciones específicas del SDK Nativo de Ollama







  ### 1. Gestión de modelos (listar e inspeccionar)

In [ ]:
def gestionar_modelos_ollama():
    """Listar modelos disponibles y mostrar información detallada"""
    
    try:
        # Listar modelos disponibles
        modelos = ollama.list()
        
        print("📋 Modelos disponibles en Ollama:")
        print("=" * 50)
        
        if not modelos['models']:
            print("❌ No hay modelos instalados")
            print("💡 Instala un modelo con: ollama run gemma2:2b")
            return None
        
        # Mostrar lista de modelos
        for modelo in modelos['models']:
            nombre = modelo['model']
            tamaño = modelo['size'] / (1024**3)  # Convertir a GB
            modificado = modelo['modified_at']
            
            print(f"{nombre} ({tamaño:.1f} GB)")
            print("-"*50)
            print(f"  📅 Modificado: {modificado}")
            print()
        
            # Obtener información específica del modelo
            info = ollama.show(modelo['model'])
            
            # Información básica
            if 'modelfile' in info:
                print("📋 Modelfile:")
                print(info['modelfile'][:200] + "..." if len(info['modelfile']) > 200 else info['modelfile'])
                print()
            
            # Parámetros
            if 'parameters' in info:
                print("⚙️ Parámetros:")
                print(info['parameters'])
                print()
            
            # Template
            if 'template' in info:
                print("📝 Template:")
                print(info['template'][:150] + "..." if len(info['template']) > 150 else info['template'])
                print()
        
        return modelos
        
    except Exception as e:
        print(f"❌ Error: {e}")
        return None

# Ejemplos de uso
gestionar_modelos_ollama()


  ### 2. Descargar modelos programáticamente

In [ ]:
def descargar_modelo_si_no_existe(nombre_modelo):
    """Descargar un modelo solo si no existe"""
    try:
        # Verificar si el modelo ya existe
        modelos = ollama.list()
        modelos_instalados = [m['model'] for m in modelos['models']]
        
        if nombre_modelo in modelos_instalados:
            print(f"✅ Modelo {nombre_modelo} ya está instalado")
            return True
        
        print(f"📥 Descargando modelo {nombre_modelo}...")
        print("⏳ Esto puede tomar varios minutos...")
        
        # Descargar el modelo
        ollama.pull(nombre_modelo)
        
        print(f"✅ Modelo {nombre_modelo} descargado correctamente")
        return True
        
    except Exception as e:
        print(f"❌ Error descargando modelo: {e}")
        return False

# Ejemplo de uso (comentado para evitar descargas no deseadas)
descargar_modelo_si_no_existe("qwen2.5:0.5b")


  ## 4. Modelos de Hugging Face en Ollama


  ### 🚀 Integración con Hugging Face

  **Formato**: `ollama run hf.co/usuario/nombre-modelo-GGUF:cuantización`

  **Buscar**: [Hugging Face Models](https://huggingface.co/models) → Filtrar por "GGUF"


  📚 **Documentación útil:**


  - [Hugging Face Models](https://huggingface.co/models)

  - [GGUF Format Guide](https://huggingface.co/docs/transformers/main/gguf)



  ### 🔍 Entendiendo las cuantizaciones

  Las cuantizaciones siguen este patrón: **Q[bits]_[método]_[variante]**

  - **Q**: Indica que es una cuantización (Quantization)

  - **Número**: Bits por peso (2, 4, 5, 6, 8) - menos bits = menor tamaño

  - **K**: Método k-quantization

  - **Sufijos**:

    - **S**: Small (menor precisión, más rápido)

    - **M**: Medium (balance entre precisión y velocidad)

    - **L**: Large (mayor precisión, más lento)

    - **_0**: Otra forma de cuantizar mas antigua
  



  💡 **Consejo práctico**: Empieza con Q4_K_M para un buen balance

  ### 🎯 Cuantizaciones más comunes

  | Cuantización | Tamaño | Calidad | Velocidad |
  |-------------|--------|---------|-----------|
  | **Q2_K** | Muy pequeño | Baja | Muy rápida |
  | **Q4_K_M** | Pequeño | Buena | Rápida |
  | **Q5_K_M** | Medio | Muy buena | Media |
  | **Q6_K** | Grande | Excelente | Lenta |
  | **Q8_0** | Muy grande | Casi perfecta | Muy lenta |



In [ ]:
def usar_modelo_huggingface(mensaje, modelo_hf="hf.co/MaziyarPanahi/gemma-3-1b-it-GGUF:Q4_K_M"):
    """Usar un modelo de Hugging Face en Ollama"""
    
    try:
        # Verificar si el modelo ya está instalado
        modelos = ollama.list()
        modelos_instalados = [m['model'] for m in modelos['models']]
        
        if modelo_hf not in modelos_instalados:
            print(f"📥 Modelo {modelo_hf} no está instalado")
            print(f"💡 Comando: ollama pull {modelo_hf}")
            print("🔄 Ejecuta el comando en terminal y luego vuelve aquí")
            return None
        
        # Usar el modelo de HF
        response = ollama.chat(
            model=modelo_hf,
            messages=[{'role': 'user', 'content': mensaje}]
        )
        
        print(f"🤗 Hugging Face responde:")
        print(response['message']['content'])
        
        return response['message']['content']
        
    except Exception as e:
        print(f"❌ Error con modelo HF: {e}")
        print(f"💡 Asegúrate de descargar el modelo: ollama run {modelo_hf}")
        return None

# Ejemplo de uso
usar_modelo_huggingface("¿Cuál es la capital de Francia?")


  ## 5. Comparación: Local vs Cloud







  ### 🎯 Objetivo de la comparación



  Comparar el rendimiento entre modelos locales y en la nube usando **el mismo modelo base** para obtener una comparación justa.







  ### 🔍 Metodología



  1. **Mismo modelo**: Usar Gemma para ambos casos (local y nube)

  2. **Misma pregunta**: Prompt idéntico para ambos modelos

  3. **Métricas consistentes**: Tiempo de respuesta, caracteres, calidad

  4. **Condiciones controladas**: Misma configuración de temperatura y tokens



In [ ]:
from pydantic import BaseModel, Field
from dotenv import load_dotenv
from google import genai    
from google.genai import types
import time

load_dotenv()

cliente_gemini = genai.Client()

class EvaluacionCalidad(BaseModel):
    """Define la estructura para la evaluación de calidad usando Pydantic."""
    razonamiento: str
    puntuacion_calidad: int = Field(default=0, ge=0, le=100, description="Puntuación entre 0 y 100")


def evaluar_calidad_respuesta(prompt, respuesta_generada):
    """
    Utiliza Gemini 2.5 Flash con un esquema Pydantic para evaluar la calidad de una respuesta.
    """
    if not cliente_gemini:
        print("❌  Cliente de Gemini no inicializado. Omitiendo evaluación.")
        return {"puntuacion_calidad": 0, "razonamiento": "Cliente no configurado."}
        
    print("\n🔎  Evaluando calidad con Gemini 2.5 Flash")
    
    evaluador_prompt = f"""
    **Tarea de Evaluación de Calidad de Respuesta de LLM**
    **Contexto:**
    - Prompt Original: "{prompt}"
    - Respuesta a Evaluar: "{respuesta_generada}"
    **Instrucciones:**
    Actúa como un experto evaluador de IA. Analiza la "Respuesta a Evaluar" basándote en el "Prompt Original".
    Evalúa según: Relevancia, Coherencia, Precisión y Completitud.
    Proporciona una puntuación y un razonamiento breve.
    """
    
    try:
        # Se utiliza el método client.models.generate_content como indicaste.
        respuesta_evaluador = cliente_gemini.models.generate_content(
            model='gemini-2.5-flash',
            contents=[types.Content(parts=[types.Part.from_text(text=evaluador_prompt)])],
            config=types.GenerateContentConfig(
                response_mime_type="application/json",
                response_schema=EvaluacionCalidad,
                temperature=0,
            )
        )
        evaluacion = respuesta_evaluador.parsed 
        print(f"✅  Evaluación completada. Puntuación: {evaluacion.puntuacion_calidad}")
        return evaluacion
    except Exception as e:
        print(f"❌  Error durante la evaluación de calidad: {e}")
        return {"puntuacion_calidad": 0, "razonamiento": "Error en la evaluación."}


def medir_y_evaluar(funcion_generadora, nombre_proveedor, prompt):
    """
    Mide el rendimiento de una función generadora y evalúa la calidad de su respuesta.
    """
    print(f"\n📊  PROBANDO: {nombre_proveedor.upper()}")
    print("-" * 50)
    
    inicio = time.time()
    try:
        respuesta_texto = funcion_generadora()
        tiempo_respuesta = time.time() - inicio
        
        fragmento = respuesta_texto[:30].strip().replace("\n", " ") + "..."

        print(f"✅  Respuesta generada en {tiempo_respuesta:.2f}s")
        print(f"📝  Fragmento: {fragmento}")

        evaluacion = evaluar_calidad_respuesta(prompt, respuesta_texto)

        return {
            'proveedor': nombre_proveedor,
            'tiempo': tiempo_respuesta,
            'Caracteres': len(respuesta_texto),
            'puntuacion_calidad': evaluacion.puntuacion_calidad,
            'razonamiento_calidad': evaluacion.razonamiento,
            'exito': True
        }
    except Exception as e:
        print(f"❌  Error generando respuesta con {nombre_proveedor}: {e}")
        return {'proveedor': nombre_proveedor, 'exito': False}


def comparar_modelos():
    """
    Función principal que orquesta la comparación de modelos.
    """
    if not cliente_gemini:
        print("\nNo se puede continuar sin la configuración del cliente de Gemini.")
        return

    prompt = """
El Reto del Triángulo Viajero
Un punto se encuentra inicialmente en el origen de un plano cartesiano, en la coordenada (0, 0). Este punto se moverá para formar los vértices de un triángulo ABC siguiendo estas reglas:

Para llegar al Vértice A: El punto se desplaza desde el origen una distancia de 10 unidades en la misma dirección y sentido que el vector (3, 4).

Para llegar al Vértice B: Desde el Vértice A, el punto se desplaza 7 unidades en la dirección negativa del eje Y (hacia abajo).

El Vértice C se queda en el origen, (0, 0).

Tu Misión:

Responde a las siguientes preguntas, mostrando tus cálculos y razonamiento en cada paso.

Calcula las coordenadas exactas de los Vértices A y B.

Calcula el perímetro del triángulo ABC.

Calcula el área del triángulo ABC.

Clasifica el triángulo (por ejemplo: isósceles, escaleno, rectángulo). Debes justificar tu respuesta basándote en las longitudes de sus lados o en sus ángulos.


"""
    print("🏁 COMPARACIÓN DE RENDIMIENTO Y CALIDAD DE MODELOS 🏁")
    print("=" * 70)

    cliente_ollama = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")

    proveedores = [
        (
            # Lambda actualizada para usar el patrón client.models.generate_content
            lambda: cliente_gemini.models.generate_content(
                model='gemma-3-1b-it',
                contents=[types.Content(parts=[types.Part.from_text(text=prompt)])],
                config=types.GenerateContentConfig(
                    temperature=0.0,
                    max_output_tokens=400
                )
            ).text,
            "Gemma 3 1B (Nube)"
        ),
        (
            lambda: cliente_ollama.chat.completions.create(
                model="hf.co/MaziyarPanahi/gemma-3-1b-it-GGUF:Q4_K_M",
                messages=[{"role": "user", "content": prompt}],
                temperature=0.0,
                max_tokens=400
            ).choices[0].message.content,
            "Gemma 3 1B (Local y Quant)"
        )
    ]

    resultados = []
    for funcion_gen, nombre in proveedores:
        resultado = medir_y_evaluar(funcion_gen, nombre, prompt)
        resultados.append(resultado)

    # --- Mostrar Resumen Comparativo ---
    print("\n🏆 RESUMEN COMPARATIVO FINAL")
    print("=" * 85)
    print(f"{'Proveedor':<30} | {'Tiempo (s)':<12} | {'Caracteres':<8} | {'Calidad (1-100)':<16} | {'Estado'}")
    print("-" * 85)

    exitosos = [r for r in resultados if r.get('exito')]
    for r in resultados:
        if r.get('exito'):
            print(f"{r['proveedor']:<30} | {r['tiempo']:<12.2f} | {r['Caracteres']:<8} | {r['puntuacion_calidad']:<16} | ✅")
        else:
            print(f"{r['proveedor']:<30} | {'--':<12} | {'--':<8} | {'--':<16} | ❌ Error")

    # --- Análisis de Resultados ---
    if len(exitosos) > 1:
        mas_rapido = min(exitosos, key=lambda x: x['tiempo'])
        mejor_calidad = max(exitosos, key=lambda x: x['puntuacion_calidad'])

        print("\n📈 ANÁLISIS DE GANADORES:")
        print(f"🚀 Más Rápido: {mas_rapido['proveedor']} ({mas_rapido['tiempo']:.2f}s)")
        print(f"⭐ Mejor Calidad: {mejor_calidad['proveedor']} (Puntuación: {mejor_calidad['puntuacion_calidad']}/10)")


if __name__ == "__main__":
    comparar_modelos()

  ## 6. Problema: Asistente Financiero Local con Mejores Prácticas


  ### 🎯 Objetivo

  Crear un **Asistente Financiero** profesional usando **orientación a objetos** que integre modelos locales y en la nube.


  ### 📋 Requisitos 

1. **Múltiples proveedores**: Implementar la capacidad de utilizar diferentes proveedores de modelos de lenguaje, incluyendo Azure OpenAI, Google Gemini y un modelo local (Ollama).

2. **Streaming en tiempo real**: Mostrar las respuestas del asistente en tiempo real, de manera que aparezcan progresivamente mientras se generan.

3. **Gestión de historial**: Mantener el contexto de la conversación almacenando hasta los últimos 10 mensajes intercambiados entre el usuario y el asistente.

4. **Comandos especiales**: Implementar los siguientes comandos especiales para controlar el comportamiento del asistente:
   - `/estadisticas`: Mostrar estadísticas de la conversación actual.
   - `/limpiar`: Limpiar el historial de la conversación.
   - `/cambiar`: Cambiar el proveedor de modelos actualmente en uso.
   - `/salir`: Terminar la conversación y salir del programa.
   - `/ayuda`: Mostrar la lista de comandos disponibles y su descripción.

5. **Estadísticas**: Realizar un seguimiento de las estadísticas de la conversación, incluyendo el conteo de mensajes enviados y la duración de la conversación.

6. **Gestión de errores**: Manejar adecuadamente los errores y excepciones que puedan ocurrir durante la ejecución del programa. Controlar las excepciones esperadas y proporcionar mensajes de error informativos al usuario. Lanzar excepciones personalizadas cuando sea necesario para un manejo de errores más granular.

7. **Encapsulamiento en clase ChatbotFinanciero**: Encapsular toda la funcionalidad del Asistente Financiero en una clase llamada `ChatbotFinanciero`. La clase debe contener los métodos necesarios para interactuar con los diferentes proveedores, gestionar el historial de la conversación, manejar los comandos especiales y realizar el seguimiento de las estadísticas.




### 6.1 Estructura base del chatbot

In [ ]:

import os
from datetime import datetime
from dotenv import load_dotenv
from openai import OpenAI, AzureOpenAI
from google import genai

# Cargar variables de entorno
load_dotenv()

# Constantes para los proveedores

class ChatbotFinanciero:
    def __init__(self):
        """Inicializar el chatbot con todas las configuraciones"""
        # TODO: Definir prompt del sistema
        # TODO: Inicializar historial de mensajes
        # TODO: Inicializar proveedor_activo
        # TODO: Inicializar estadísticas
        pass

    def responder_azure(self, mensajes: list[dict]) -> str:
        """Responde usando Azure OpenAI con streaming"""
        # TODO: Crear cliente AzureOpenAI
        # TODO: Hacer llamada con streaming
        # TODO: Imprimir respuesta en tiempo real
        pass

    def responder_gemini(self, mensajes: list[dict]) -> str:
        """Responde usando Google Gemini con streaming"""
        # TODO: Crear cliente genai
        # TODO: Convertir mensajes a formato Gemini
        # TODO: Hacer llamada con streaming
        # TODO: Imprimir respuesta en tiempo real
        pass

    def responder_ollama(self, mensajes: list[dict]) -> str:
        """Responde usando Ollama con streaming"""
        # TODO: Crear cliente OpenAI apuntando a localhost:11434
        # TODO: Hacer llamada con streaming
        # TODO: Imprimir respuesta en tiempo real
        pass

    def configurar_proveedor(self, proveedor: str) -> None:
        """Configura el proveedor a usar"""
        # TODO: Configurar proveedor_activo 
        pass

    def responder(self, mensaje: str) -> str:
        """Método principal - responde usando el proveedor configurado"""
        # TODO: Verificar si hay un proveedor configurado, lanzar excepción ValueError si no lo hay
        # TODO: Añadir mensaje al historial
        # TODO: Llamar a la función correcta según proveedor_activo
        # TODO: Gestionar historial (limitar a 10 mensajes)
        pass

    def mostrar_estadisticas(self) -> None:
        """Muestra estadísticas de la conversación"""
        # TODO: Calcular duración, contar mensajes, mostrar métricas
        pass

    def limpiar_historial(self) -> None:
        """Limpia el historial de mensajes"""
        # TODO: Limpiar historial
        pass

    def cambiar_proveedor(self) -> None:
        """Cambia el proveedor activo"""
        # TODO: Preguntar al usuario por el nuevo proveedor
        # TODO: Llamar a configurar_proveedor() con el nuevo proveedor
        pass

    def mostrar_ayuda(self) -> None:
        """Muestra la ayuda con los comandos disponibles"""
        # TODO: Imprimir lista de comandos y su descripción
        pass

def main() -> None:
    """Función principal para el chatbot interactivo"""
    # TODO: Crear instancia del chatbot
    # TODO: Configurar proveedor inicial
    # TODO: Mostrar comandos disponibles
    # TODO: Bucle principal del chat
    # TODO: Manejar comandos especiales (/estadisticas, /limpiar, /cambiar, /ayuda, /salir)
    # TODO: Enviar mensaje y obtener respuesta
    # TODO: Manejar excepciones y mostrar mensajes de error al usuario
    pass

# Crear una instancia del chatbot
chatbot = ChatbotFinanciero()
# Cambiar proveedor
chatbot.cambiar_proveedor()
# Responder a un mensaje
respuesta = chatbot.responder("¿Cuál es la capital de España?")



  ## 🎉 ¡Felicitaciones!

  Has completado el lab de **Modelos de IA Locales con Ollama y LM Studio**. Ahora tienes las habilidades para:

  ✅ Ejecutar modelos de IA localmente

  ✅ Gestionar modelos con herramientas profesionales

  ✅ Integrar modelos locales en aplicaciones